# Calibration silver bullet spectrometer

In the cell below, the calibration data (both wavelength and absolute calibration) from the silver bullet spectrometer is processed (more information see this Youtube video: https://youtu.be/WP0HZCWyedU). In addition, 5 plotting functions are defined, which you can choose after running the cell below.

The plotting functions are:
* Plot_raw
* Outlier_report_peirce
* Plot_norm
* Plot_std
* Plot_std_variation

If you want to plot the data for specific years (can be 1 or several), input the year as a list variable (e.g. Plot_raw([2021,2022]) )
Otherwise, if you want to plot all available years, you can use: Plot_raw(all_years)

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import statistics
import scipy.special
from sklearn.metrics import r2_score
import time

##FUNCTIONS
from calibration_functions import Read_Sorted, Adjustment, peirce_dev, Peirce_crit, Average, Bg_subtraction, Cal_fact, Plot_raw, Outlier_report_peirce, Outlier_report_peirce_std,  Plot_norm, Plot_std, Plot_std_variation, interpol

##Lamp
Lamp_200W_B_range_of_interest=[456129,465687,473653,481425,492220,498682]
Lamp_200W_wavelength_range_of_interest=np.arange(8200,8800,100)
fiti=np.polyfit(Lamp_200W_wavelength_range_of_interest,Lamp_200W_B_range_of_interest,1,rcond=None, full=True)
def synthetic_fit(x):
	return(fiti[0][0]*x+fiti[0][1])
score=r2_score(Lamp_200W_B_range_of_interest,
               synthetic_fit(Lamp_200W_wavelength_range_of_interest))   
    
##INPUTS AND VARIABLES
##In this section, input the files and parameters
scan_start=8285
scan_stop=8705
nb_scan=381
r=2
rho=0.98
scanning_wl=np.linspace(scan_start,scan_stop,nb_scan,True)

path_data_2011_bg=("./calibration_files/CalData/s0202112_background_for_abs.lyr")
path_data_2011_abs=("./calibration_files/CalData/s0202112_abscal.lyr")
path_cal_2011=("./calibration_files/WavelengthCalibrations/Wavelength-Feb2011.Cal")
R_2011=51.85 #m
a_2011=44 #deg

path_data_2013_bg=("./calibration_files/CalData/s0602132Silver_BG.lyr")
path_data_2013_abs=("./calibration_files/CalData/s0602132Silver_ABS.lyr")
path_cal_2013=("./calibration_files/WavelengthCalibrations/Wavelength-Feb2013.Cal")
R_2013=52.05 #m
a_2013=43.5 #deg

path_data_2014_bg=("./calibration_files/CalData/silver_Background_0502142.lyr")
path_data_2014_abs=("./calibration_files/CalData/silver_ABS_CAL_0502142.lyr")
path_cal_2014=("./calibration_files/WavelengthCalibrations/Wavelength-Feb2014.Cal")
R_2014=51.87 #m
a_2014=45 #deg

path_data_2015_bg=("./calibration_files/CalData/s1202152BackgroundScans.lyr")
path_data_2015_abs=("./calibration_files/CalData/s1202152ABS_CAL.lyr")
path_cal_2015=("./calibration_files/WavelengthCalibrations/Wavelength-Feb2015.Cal")
R_2015=51.88 #m
a_2015=46 #deg

path_data_2016_bg=("./calibration_files/CalData/Back_10022016_silver_s1002161.lyr")
path_data_2016_abs=("./calibration_files/CalData/ABS_CAL_10022016_silver_s1002161.lyr")
path_cal_2016=("./calibration_files/WavelengthCalibrations/Wavelength-Feb2016.Cal")
R_2016=51.93 #m
a_2016=45 #deg

path_data_2017_bg=("./calibration_files/CalData/s0102172_BACKGROUND.lyr")
path_data_2017_abs=("./calibration_files/CalData/s0102172_ABS_CAL.lyr")
path_cal_2017=("./calibration_files/WavelengthCalibrations/Wavelength-Feb2017.Cal")
R_2017=51.974 #m
a_2017=44 #deg

path_data_2018_bg=("./calibration_files/CalData/s0802182_BACKGROUND.lyr")
path_data_2018_abs=("./calibration_files/CalData/s0802182_ABS_CAL.lyr")
path_cal_2018=("./calibration_files/WavelengthCalibrations/Wavelength-Feb2018.Cal")
R_2018=51.92 #m
a_2018=44 #deg

path_data_2019_bg=("./calibration_files/CalData/DARK_SCAN_ABS_CAL_s0702192.lyr")
path_data_2019_abs=("./calibration_files/CalData/ABS_CALs0702192.lyr")
path_cal_2019=("./calibration_files/WavelengthCalibrations/Wavelength-Feb2019.Cal")
R_2019=52.02 #m
a_2019=45.2 #deg

path_data_2020_bg=("./calibration_files/CalData/s2002202_CAL_BACKGROUND.lyr")
path_data_2020_abs=("./calibration_files/CalData/s2002202_ABS_CAL.lyr")
path_cal_2020=("./calibration_files/WavelengthCalibrations/Wavelength-Jan2020.Cal")
R_2020=51.98 #m
a_2020=45 #deg

path_data_2021_bg=("./calibration_files/CalData/s1802212_BG_AbsCal.lyr")
path_data_2021_abs=("./calibration_files/CalData/s1802212_ABS_cal.lyr")
path_cal_2021=("./calibration_files/WavelengthCalibrations/Wavelength-Feb2021.Cal")
R_2021=51.95 #m
a_2021=45.5 #deg

path_data_2022_bg=("./calibration_files/CalData/s1002222_BG_AbsCal.lyr")
path_data_2022_abs=("./calibration_files/CalData/s1002222_ABS_cal.lyr")
path_cal_2022=("./calibration_files/WavelengthCalibrations/Wavelength-Feb2022.Cal")
R_2022=51.96 #m
a_2022=45  #deg

dico=dict(a2011=[path_data_2011_bg,path_data_2011_abs,path_cal_2011],
          a2013=[path_data_2013_bg,path_data_2013_abs,path_cal_2013],
          a2014=[path_data_2014_bg,path_data_2014_abs,path_cal_2014],
          a2015=[path_data_2015_bg,path_data_2015_abs,path_cal_2015],
          a2016=[path_data_2016_bg,path_data_2016_abs,path_cal_2016],
          a2017=[path_data_2017_bg,path_data_2017_abs,path_cal_2017],
          a2018=[path_data_2018_bg,path_data_2018_abs,path_cal_2018],
          a2019=[path_data_2019_bg,path_data_2019_abs,path_cal_2019],
          a2020=[path_data_2020_bg,path_data_2020_abs,path_cal_2020],
          a2021=[path_data_2021_bg,path_data_2021_abs,path_cal_2021],
          a2022=[path_data_2022_bg,path_data_2022_abs,path_cal_2022],
          )
dicop=dict(p2011=[R_2011,a_2011],p2013=[R_2013,a_2013],p2014=[R_2014,a_2014],
           p2015=[R_2015,a_2015],p2016=[R_2016,a_2016],p2017=[R_2017,a_2017],
           p2018=[R_2018,a_2018],p2019=[R_2019,a_2019],p2020=[R_2020,a_2020],
           p2021=[R_2021,a_2021],p2022=[R_2022,a_2022]
           )
all_years=[2011,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]

An example of plotting:

In [ ]:
#Plot_raw(all_years)
#Outlier_report_peirce(all_years)
Plot_norm(all_years)
#Plot_std([2022,2016])
#Plot_std_variation(all_years)